In [25]:
import pandas as pd
import re
average_path_1 = "tpc-h/result_log/result_log_5GB/time_1/average_query_sys_params.csv"
average_path_2 = "tpc-h/result_log/result_log_5GB/time_2/average_query_sys_params.csv"
average_path_3 = "tpc-h/result_log/result_log_5GB/time_3/average_query_sys_params.csv"

worst_path_1 = "tpc-h/result_log/result_log_5GB/time_1/worst_query_sys_params.csv"
worst_path_2 = "tpc-h/result_log/result_log_5GB/time_2/worst_query_sys_params.csv"
worst_path_3 = "tpc-h/result_log/result_log_5GB/time_3/worst_query_sys_params.csv"

worst_output_path = "tpc-h/result_log/result_log_5GB/worst_query_case_5GB.csv"
average_output_path = "tpc-h/result_log/result_log_5GB/average_query_case_5GB.csv"
# Đường dẫn đến 3 file CSV
average_paths = [
    average_path_1,average_path_2,average_path_3
]
worst_paths = [
    worst_path_1,worst_path_2,worst_path_3
]
# Các cột cần thiết
columns_to_use = [
    "query_id","cpu_used(%)","ram_used(gb)","time(ms)"
]


In [26]:
def process_query_metrics(paths, columns_to_use, output_path):
    # Read, filter, and clean data
    dfs = [
        pd.read_csv(path, comment='/')[columns_to_use]
        .assign(query_id=lambda df: df['query_id'].astype(str).str.strip())  # Convert to string and strip
        for path in paths
    ]
    
    # Merge data and calculate average by query_id
    all_data = pd.concat(dfs)
    average_metrics = all_data.groupby('query_id', as_index=False).mean(numeric_only=True)
    average_metrics = average_metrics.round(2)
    
    # Add column for sorting by number in query_id
    average_metrics['query_number'] = average_metrics['query_id'].apply(lambda x: int(re.search(r'\d+', x).group()))
    
    # Sort by query number
    average_metrics = average_metrics.sort_values(by='query_number').drop(columns=['query_number'])
    
    # Write to file
    average_metrics.to_csv(output_path, index=False)
    
    return average_metrics

In [27]:
# Process average metrics
average_metrics = process_query_metrics(
    average_paths, 
    columns_to_use, 
    average_output_path
)

# Process worst metrics
worst_metrics = process_query_metrics(
    worst_paths,
    columns_to_use,
    worst_output_path
)